In [125]:
import requests
import telebot
import moment
from datetime import datetime
from dotenv import dotenv_values
config = dotenv_values("../.env")




In [126]:

# mydb = mysql.connector.connect(
#     host=config["DBHOST"],
#     user=config["DBUSER"],
#     password=config["DBPASS"],
#     database=config["DBNAME"]
# )


# mycursor = mydb.cursor(dictionary=True)

# mycursor.execute("SELECT * from countries WHERE name = 'Andorra'")

# myresult = mycursor.fetchall()

# print(myresult)
# for iso in flags:
#   sql = f"UPDATE countries SET flag_emoji = '{flags[iso]['flag']}' WHERE iso2 = '{iso.upper()}'"
#   mycursor.execute(sql)

# mydb.commit()



In [146]:

def __fetch_fixture_by_id(fixture_id):
    # CODE SHOULD BE CHANGED TO FINDING THE FIXTURE FROM DYNAMODB
    res = requests.get(
        "https://gist.githubusercontent.com/VishalBty/c3bd3bae0d4fe8e1a6ccaae1e5f7dc0c/raw/dd4eb5597452131ff3a782980151bd14675e76ba/fixture.json")
    return res.json()


def __fetch_strategy_by_id(strategy_id):
    # CODE SHOULD BE CHANGED TO FINDING THE STRATEGY FROM MYSQL
    # SENDING ALERT SHOULD BE ABORTED IF STRATEGY IS NOW INACTIVE
    # SELECT * from strategies WHERE id=1 AND active=true
    # HAS MANY ITEMS
    # select `code`, `category`, `overall`, `home`, `label`, `away`, `strategy_prematch_rules`.* from `strategy_prematch_rules` inner join `rules` as `rule` on `rule`.`uid` = `strategy_prematch_rules`.`uid` where `strategy_prematch_rules`.`strategy_id` in (?, ?, ?, ?, ?, ?, ?)
    # select `strategy_inplay_rules`.`strategy_id`, `first_rule`.`code` as `first_code`, `second_rule`.`code` as `second_code`, `first_category`, `second_category`, `comparator`, `value`, `first_team`, `second_team` from `strategy_inplay_rules` inner join `rules` as `first_rule` on `first_rule`.`uid` = `strategy_inplay_rules`.`first_rule_id` inner join `rules` as `second_rule` on `second_rule`.`uid` = `strategy_inplay_rules`.`second_rule_id` where `strategy_inplay_rules`.`strategy_id` in (?, ?, ?, ?, ?, ?, ?)
    # select `code`, `label`, `strategy_outcomes`.* from `strategy_outcomes` inner join `outcomes` as `outcome` on `outcome`.`id` = `strategy_outcomes`.`outcome_id` where `strategy_outcomes`.`strategy_id` in (?, ?, ?, ?, ?, ?, ?)
    res = requests.get(
        "https://gist.githubusercontent.com/VishalBty/7d82150efee726414a0b0b5e1d31bc61/raw/733350ba8a0befd2bf98c63908224c0b3aebac6d/strategy.json")
    return res.json()


def __fetch_user_by_id(user_id):
    # CODE SHOULD BE CHANGED TO FINDING THE USER FROM MYSQL
    # REQUIRED TO FETCH THE TELEGRAM CHAT ID OF A USER
    # SELECT * from users RIGHT JOIN subscriptions ON subscriptions.email = users.email AND status='active' AND end_date > NOW() WHERE users.id=1 AND telegram IS NOT NULL 
    return {"telegram": 342833991}


def __field_for_probability(rule):
    return f'probability.{rule["code"]}'


def __field_for_odds(rule):
    return f'pre_odds.{rule["code"]}'


def __field_for_team(rule, team):
    return f'{team}.{rule[rule["location"]]}'


def __field_for_aggregate_stats(rule):
    if rule["team"] == "home":
        field = __field_for_team(rule, "home")
    elif rule["team"] == "away":
        field = __field_for_team(rule, "away")
    else:
        field = [__field_for_team(rule, "home"),
                 __field_for_team(rule, "away")]
    return field


def __format_stats_as_telegram_message(pre_match_data):
    lines = []
    for item in pre_match_data:
        value = item["value"]
        label = item["label"]
        line = f"<b>{label}</b>: {value}"
        lines.append(line)
    return "\n".join(lines)


def __get_prematch_data(fixture, strategy):
    pre_match_data = []
    locations = {
        "home": "Home",
        "away": "Away",
        "overall": "Overall",
    }
    teams = {
        "home": "Home",
        "away": "Away",
        "both": "Both",
    }
    for rule in strategy["strategy_prematch_rules"]:
        label = rule["label"]
        if rule["category"] == "probability":
            label += " (Probability)"
            fields = __field_for_probability(rule)
        elif rule["category"] == "odds":
            label += " (Odds)"
            fields = __field_for_odds(rule)
        else:
            location = locations[rule["location"]]
            team = teams[rule["team"]]
            label += f" [{team}] ({location})"
            fields = __field_for_aggregate_stats(rule)

        if not isinstance(fields, list):
            field = fields
        else:
            field = fields[0]
        value = fixture[field]
        pre_match_data.append({"value": value, "label": label})
    return pre_match_data


def __get_inplay_data(fixture):
    in_play_data = []
    stats = {
        "goals": "Goals",
        "corners": "Corners",
        "shots_off_target": "Shots Off Target",
        "shots_on_target": "Shots On Target",
        "shots": "Shots",
        "yellowcards": "Yellow Cards",
        "redcards": "Red Cards",
        "penalties": "Penalties",
        "dangerous_attacks": "Dangerous Attacks",
        "attacks": "Attacks",
        "possession": "Possession",
        "substitutions": "Substitutions"
    }
    for stat, label in stats.items():
        home_value = fixture[f"stats.home.{stat}"]
        away_value = fixture[f"stats.away.{stat}"]
        value = f"{home_value} - {away_value}"
        in_play_data.append({"value": value, "label": label,
                            "home": home_value, "away": away_value})
    return in_play_data


def __create_pre_match_message(fixture, strategy, utcoffset=0, locale="en-gb"):

    outcomes_text = "\n".join(map(lambda x: x["label"], strategy["outcomes"]))
    league_name = fixture["league_name"]
    flag_emoji = fixture["flag_emoji"]
    country_name = fixture["country_name"]
    league_text = f"{flag_emoji or '🏆'} <b>{country_name} {league_name}</b>"

    title = strategy["title"]
    home_name = fixture["home_name"]
    away_name = fixture["away_name"]
    note = strategy["note"]
    unix_timestamp = fixture["timestamp"] * 1000

    starts_in = strategy["timer"]["minute"]
    match_time = moment.unix(
        unix_timestamp, utc=True).strftime("%d %b %Y %I:%M")
    match_timer_text = f"""⏱️ <b>Match Time:</b> {match_time} (Starts in {starts_in} mins)"""

    stats = __get_prematch_data(fixture, strategy)
    stats_message = __format_stats_as_telegram_message(stats)

    pre_match_1x2_odds = f"""<b>1X2 Pre-Match Odds:</b>\n{fixture["pre_odds.home_win"]} {fixture["pre_odds.draw"]} {fixture["pre_odds.away_win"]}"""

    lines = [
        f"🔔 <b>{title}</b>",
        "",
        f"🆚 <b>{home_name}</b> v <b>{away_name}</b>",
        league_text,
        "",
        match_timer_text,
        "",
        f"<b>Desired Outcomes:</b>\n{outcomes_text}",
        "",
        stats_message,
        "",
        pre_match_1x2_odds
    ]
    if note:
        note_text = f"\n<b>💭 My Note</b>\n{note}"
        lines.append(note_text)

    message = "\n".join(lines)
    return message


def __create_in_play_message(fixture, strategy, utcoffset=0, locale="en-gb"):

    outcomes_text = "\n".join(map(lambda x: x["label"], strategy["outcomes"]))
    #   .map(outcome => capitalize(outcome.split("_").join(" ")))
    #   .join(",");
    league_name = fixture["league_name"]
    flag_emoji = fixture["flag_emoji"]
    country_name = fixture["country_name"]
    league_text = f"{flag_emoji or '🏆'} <b>{country_name} {league_name}</b>"

    goal_events = sorted(filter(
        lambda x: x["type"] == 'goal', fixture["events"]), key=lambda x: x["minute"]+(x["extra_minute"] or 0))

    title = strategy["title"]
    home_name = fixture["home_name"]
    away_name = fixture["away_name"]
    note = strategy["note"]

    minute = fixture["minute"]
    match_timer_text = f"""<b>⏱️ Timer:</b> {minute}'"""

    last_goal_text = "None"
    if goal_events:
        last_goal = goal_events[-1]
        team_id = last_goal["team_id"]
        team = "Home" if team_id == fixture["home_id"] else "Away"
        goal_minute = last_goal["minute"]
        from_now = minute - goal_minute
        last_goal_text = f"{team} at {goal_minute}’ ({from_now} mins ago)"

    stats = __get_prematch_data(fixture, strategy)
    stats_message = __format_stats_as_telegram_message(stats)

    livestats = __get_inplay_data(fixture)
    livestats_message = __format_stats_as_telegram_message(livestats)

    pre_match_1x2_odds = f"""<b>1X2 Pre-Match Odds:</b>\n{fixture["pre_odds.home_win"]} {fixture["pre_odds.draw"]} {fixture["pre_odds.away_win"]}"""
    in_play_1x2_odds = f"""<b>1X2 Live Odds:</b>\n{fixture["live_odds.home_win"]} {fixture["live_odds.draw"]} {fixture["live_odds.away_win"]}"""

    lines = [
        f"🔔 <b>{title}</b>",
        "",
        f"🆚 <b>{home_name}</b> v <b>{away_name}</b>",
        league_text,
        "",
        match_timer_text,
        f"<b>🥅 Last Goal:</b> {last_goal_text}",
        "",
        f"<b>Desired Outcomes:</b>\n{outcomes_text}",
        "",
        livestats_message,
        "",
        stats_message,
        "",
        pre_match_1x2_odds,
        in_play_1x2_odds,

    ]
    if note:
        note_text = f"\n<b>💭 My Note</b>\n{note}"
        lines.append(note_text)

    message = "\n".join(lines)
    return message


def __send_telegram_alert(chat_id, message):
    TOKEN = config["TELEGRAMBOTTOKEN"]
    bot = telebot.TeleBot(TOKEN)
    return bot.send_message(chat_id, message,  parse_mode="HTML")


In [147]:

fixture_id = 18213330
strategy_id = 35
user_id = 12

user =  __fetch_user_by_id(user_id)
if not user:
  exit()
strategy = __fetch_strategy_by_id(strategy_id)
if not strategy:
  exit()
fixture = __fetch_fixture_by_id(fixture_id)
if not fixture:
  exit()
chat_id = user["telegram"]

In [148]:
message = __create_pre_match_message(fixture, strategy)
__send_telegram_alert(chat_id=chat_id, message=message)
message = __create_in_play_message(fixture, strategy)
__send_telegram_alert(chat_id=chat_id, message=message)

In [149]:
from IPython.core.display import display, HTML

message = __create_in_play_message(fixture, strategy)
display(HTML('<pre>'+message+'</pre>'))

In [150]:
message = __create_pre_match_message(fixture, strategy)
display(HTML('<pre>'+message+'</pre>'))